# Area statistics for protected areas in mainland Norway

[![colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ac-willeke/pygdal-geo-engineer/blob/main/notebooks/2024.01_conservation_and_preservation.ipynb) [![github](https://img.shields.io/badge/GitHub-View%20on%20GitHub-blue?logo=github)](https://github.com/ac-willeke/pygdal-geo-engineer/blob/main/notebooks/2024.01_conservation_and_preservation.ipynb)

**Author**: Willeke A'Campo

**Description:** This notebook is designed to compute area statistics for protected regions within mainland Norway. It specifically targets two datasets: **Verneområder** og **Foreslatte verneområder**, both provided by the Norwegian Environment Agency. In addition area statistics are calcualted for the whole of norway and for each regional zone. This allows for a comparative analysis beteween the protected areas and the overal Norwegian Territory. 

The attributes calculated can be divided into three groups:

1. Area variables for the protected areas:
    - Area (terrestrial and marine)
    - Perimeter (terrestrial and marine)
    - Land area (terrestrial)
    - Perimeter land area (terrestrial)

2. Overlay statistics for the protected area and the following datasets:
    - AR50 - Bonitet
        - Area proportion of land cover quality classes 
    - Bioklimatiske soner
        - Area proportion of bioclimatic zone classes
    - Infrastrukturindeks: 
        - Mean value of infrastructure index per protected area 
        - Area proportion of infrastructure index classes
    - Høydelag
        - Area proportion of elevation classes 

3. Spatial indices for the protected areas:
    - Density of protected area per 10x10km (SSB grid)
    - Shape Index for each protected area (land + marine)
    - Shape index for each protected area (land)






# Configure Notebook

In [1]:
# Variables to control the execution of the workflow
gis_server = True # True = use GIS server, False = use local files
load_data = False # True = load data from source, False = data is already loaded
test_area = False # True = test area, False = full area
prepare_data = False
prepare_duckdb = False # True = prepare duckdb, False = duckdb is already prepared
run_infra = True
run_ar50 = True # True = run AR50 area calc , False = AR50 is already run
run_bioklima = True 
run_shape_index = True

# Set the statistical zone: foreslatt_vern, naturvernomrade, norge_landareal_n50, region
statistical_zone = "foreslatt_vern"

if statistical_zone == "foreslatt_vern":
    sz_land = "f_vern_land"
    sz_sea = "f_vern_sjo"
    sz_land_sea = "f_vern_land_sjo"
    sz_land_vars = "foreslatt_vern_overlapp"
    
if statistical_zone == "naturvernomrade":
    sz_land = "vern_land"
    sz_sea = "vern_sjo"
    sz_land_sea = "vern_land_sjo"
    sz_land_vars = "naturvernomrade_overlapp"


In [2]:
import os 
from osgeo import gdal
from pathlib import Path
from itertools import islice

import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import box

# mapping libraries
# https://leafmap.org/faq/#how-to-use-a-specific-plotting-backend
import leafmap.foliumap as leafmap

In [3]:
project_path= Path.cwd().parents[0]
shell_path= os.path.join(project_path, "src", "shell_scripts")
python_path= os.path.join(project_path, "src", "py_scripts")

print(f"Gdal Version: {gdal.__version__} ")
print(f"Project Path: {project_path}")
print(f"Path to shell scripts: {shell_path}")
print(f"Path to python scripts: {python_path}")

Gdal Version: 3.8.4 
Project Path: /home/NINA.NO/willeke.acampo/git/pygdal-geo-engineer
Path to shell scripts: /home/NINA.NO/willeke.acampo/git/pygdal-geo-engineer/src/shell_scripts
Path to python scripts: /home/NINA.NO/willeke.acampo/git/pygdal-geo-engineer/src/py_scripts


In [4]:
# import local python scripts
from py_scripts.my_ogr import *
from py_scripts.my_duckdb import *
from py_scripts.raster import * 
from py_scripts.config import load_catalog, load_parameters
from py_scripts import decorators as dec

# Data pre-processing

------------
**Data folder structure**

```bash
|-- data
|   |-- raw                                          # single raw files received or downloaded 
|   |-- interim
|   |    |-- vern_og_bevaring.db                     # DuckDB database with (pre-processed) data layers for spatial sql analysis (ca. 30 GB)
|   |    |-- vern_25833.gpkg                         # GPKG for viewing protected area data in GIS-software
|   |    |-- bevaring_25833.gpkg                     # GPKG for viewing environment data in GIS. 
|   |    |-- admin_25833.gpkg                        # GPKG for administrative data in GIS. 
|   |-- processed
|   |    |-- vern_og_bevaring.gpkg                   # GPKG with resulting data layers 
```
------------

<br>

**Data Pipeline**

1. Raw data is downloaded and stored into the `data\raw` folder. 
2. The raw files are converted (or if possible directly imported) into the GPKGs located in `data\interim`. These GPKGS are used to view the data in GIS-software before analysis. 
3. The data layers in the GPKG are loaded into a DuckDB database and/or a (geo)dataframe for spatial analysis. 
4. The results are exported to a GPKG located in `data\processed`. 
5. Raw and Interim files can be deleted by setting the variables: *delete_raw* and *delete_interim* to True



<br>

**GeoPackage information**

**INTERIM:**

/interim/*vern_25833.gpkg*

| Layer | Dataset Name | Description | Year | Source |
|-------| ------------ | ----------- | ---- | ------ |
| naturvernomrade | Naturvernområder| Nature protected areas | 2024 | [Miljødirektoratet](https://kartkatalog.miljodirektoratet.no/Dataset/Details/0) |
| foreslatt_vern | Foreslåtte naturvernområder | Planned nature protected areas | 2024 | [Miljødirektoratet](https://kartkatalog.miljodirektoratet.no/Dataset/Details/1)| 

- *naturvernomrade* and *foreslatt_vern* imported from arcgis rest server into GPKG using ogr2ogr

/interim/*bevaring_25833.gpkg*

|Layer| Dataset Name | Description | Year | Source |
|-----| ------------ | ----------- | ---- | ------ |
| ar50_flate | AR50 | Land cover classes | 2022 | [NIBIO](https://kart8.nibio.no/nedlasting/dashboard) |
| soner_2017_1km | Bioklimatiske soner | Bioclimatic zones stored in 1x1 km grid | 2017 | [Artsdatabanken](https://data.artsdatabanken.no/Natur_i_Norge/Natursystem/Beskrivelsessystem/Regional_naturvariasjon/Bioklimatisk_sone) |
| Infra25m | Infrastrukturindeks | Infrastructure index (25m) raster converted to vector datasett. | 2024 | Internal NINA datasett |

- *ar50_2022* stored on NINA's server. Manually loaded into GPKG
- *bioklima_soner_2017* received from V. Bakkestuen (NINA) manually loaded into GPKG. 
- *Infra25m* received from V. Bakkestuen (NINA) converted to vector and lmanually oaded into GPKG.

/interim/*admin_25833.gpkg*

| Filename | Dataset Name | Description | Year | Source |
|----------| ------------ | ----------- | ---- | ------ |
| fylke_2024 | Fylker, 2024 | Provincial Boundaries | 2024 | [SSB](https://www.ssb.no/en/kart/griddata) |
| norge_havareal_ar50 | Norge havareal | Norwegian sea area based on AR50 (Arealtype = hav) | 2022 | [NIBIO](https://kart8.nibio.no/nedlasting/dashboard)|
| norge_landareal_ar50 | Norge landareal | Norwegian land area based on AR50 (Arealtype =! hav) | 2022 | [NIBIO](https://kart8.nibio.no/nedlasting/dashboard)|
| norge_ruter_10km | SSB rutenett (10x10 km)| Grid for Norway | 2024 | [SSB](https://kart.ssb.no/) |
| region_2024 | Regioner, 2024 | Regional zones: Nord, Midt, Sør, Øst, Vest based on Provincial boundaries | 2024 |  [SSB](https://www.ssb.no/en/kart/griddata) |

**PROCESSED**

/processed/*vern_og_bevaring_25833.gpkg*

| Layer | Description | Year | Source |
|-------| ----------- | ---- | ------ |
| naturvernomrade | Nature protected areas containing attributes for (i) area statistics for  AR50, Bioclimatic zones, Infrastructureindex (ii) shape index. | 2024 | Project-folder, not published |
| foreslatt_vern | Planned nature protected areas containing attributes for (i) area statistics for  AR50, Bioclimatic zones, Infrastructureindex (ii) shape index. | 2024 | Project-folder, not published |
| fastland_norge | Norway mainland area containing attributes for area statistics for AR50, Bioclimatic zones, Infrastructureindex | 2024 | Project-folder, not published |
| region | Regional zones containing attributes for area statistics for AR50, Bioclimatic zones, Infrastructureindex | 2024 | Project-folder, not published |




In [5]:
# init catalog and params
catalog = load_catalog()
params = load_parameters()

# load paths from catalog
data_path = catalog["project_data"]["filepath"]
raw_data = os.path.join(data_path, "raw")
interim_data = os.path.join(data_path, "interim")

admin_gpkg = os.path.join(data_path, "interim", "admin_25833.gpkg")
vern_gpkg = os.path.join(data_path, "interim", "vern_25833.gpkg")
bevaring_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
vern_og_bevaring_gpkg = os.path.join(data_path, "interim", "vern_og_bevaring_25833.gpkg")
db_path = os.path.join(data_path, "processed", "bevaring.db")

print(f"Path to data: {data_path}")

Path to data: /home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data


## Download Data

### Vector Data
**Import data into `admin_25833.gpkg`**

Note Norwegians's Land Area is calculated from Norways National Map N50. Please check the *norge_landareal_n50* in GIS software to see if the layer is correctly clipped! 

In [6]:
if load_data:
    
    # load paths from catalog    
    fylke_path = catalog["fylke"]["filepath"]
    fylke_lyr = catalog["fylke"]["layer"]
    ar50_path = catalog["ar50"]["filepath"]
    ar50_lyr = catalog["ar50"]["layer"]
    n50_path = catalog["n50"]["filepath"]
    n50_lyr = catalog["n50"]["layer"]

    # RUTER 10KM NORGE (SSB)
    # download from ssb, unzip, convert to gpkg, remove shapefile and zip
    ! curl -o Ruter_10KM_norge.zip "https://www.ssb.no/natur-og-miljo/_attachment/375082?_ts=1685c0e69b8"
    ! unzip Ruter_10KM_norge.zip 
    ! ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln norge_ruter_10km -nlt MULTIPOLYGON $admin_gpkg $raw_data/"Ruter_10km_Norge.shp" -lco "OVERWRITE=YES"
    ! find $raw_data -type f -name "Ruter_10km_Norge.*" -exec echo "Removing {}" \; -exec rm {} \;
    ! rm Ruter_10KM_norge.zip
    
    # FYLKE
    ! ogrinfo "$fylke_path"
    ! ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln "$fylke_lyr" -nlt MULTIPOLYGON "$interim_data/admin_25833.gpkg" "$fylke_path" "$fylke_lyr"
    
    # NORGE
    # dissolve all fylke to one feature
    ! ogr2ogr -append -f "GPKG" -dialect sqlite -sql "SELECT ST_Union(Shape) AS geom, 'Norge' AS fylkesnavn FROM $fylke_lyr" -t_srs EPSG:25833 -nln "norge" -nlt MULTIPOLYGON "$interim_data/admin_25833.gpkg" "$fylke_path"
    
    # AR50 HAVFLATE
    # import AR50 where artype = 82 and dissolve to one feature (Simplify by 1 m)
    ! ogr2ogr -append -f "GPKG" -dialect sqlite -sql "SELECT ST_Union(Shape) AS geom, artype FROM $ar50_lyr WHERE artype = '82'" -t_srs EPSG:25833 -nln "norge_havflate_ar50" -nlt MULTIPOLYGON "$interim_data/admin_25833.gpkg" "$ar50_path"
    
    # N50 HAVFLATE
    # where objtype = Havflate and dissolve to one feature
    ! ogr2ogr -append -f "GPKG" -dialect sqlite -sql "SELECT ST_Union(Shape) AS geom, objtype FROM $n50_lyr WHERE objtype = 'Havflate'" -t_srs EPSG:25833 -nln "norge_havflate_n50" -nlt MULTIPOLYGON "$interim_data/admin_25833.gpkg" "$n50_path"
    
    # N50 LANDAREAL
    remake = False
    
    if remake:
        # norge_landareal_n50: norge - norge_havflate_n50
        ! ogr2ogr -append -f "GPKG" -dialect sqlite -sql "SELECT ST_Difference(norge.geom, norge_havflate_n50.geom) AS geom, 'land' AS objtype FROM norge, norge_havflate_n50" \
            -t_srs EPSG:25833 -nln "norge_landareal_n50" -nlt MULTIPOLYGON admin_25833.gpkg admin_25833.gpkg
    else: 
        # copy layer from raw data
        ! ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln "norge_landareal_n50" -nlt MULTIPOLYGON "$interim_data/admin_25833.gpkg" "$raw_data/N50_2024_hav_landareal_25833.gpkg" "norge_landareal_n50"

**Import data into `vern_25833.gpkg`**

In [7]:
if load_data: 
    # download data from ArcGIS REST directly into the interim folder
    os.chdir(os.path.join(data_path, "interim"))
    print(os.getcwd())
        
    # ArcGIS REST: Naturvern (02.2024)
    ! ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln naturvernomrade -nlt MULTIPOLYGON vern_25833.gpkg \
        "https://kart.miljodirektoratet.no/arcgis/rest/services/vern/mapserver/0/query?where=1%3D1&outfields=*&f=json&resultRecordCount=1"

    # ArcGIS REST: Foreslåtte verneområder (02.2024)
    ! ogr2ogr -append -f "GPKG" -t_srs EPSG:25833 -nln foreslatt_vern -nlt MULTIPOLYGON vern_25833.gpkg \
        "https://kart.miljodirektoratet.no/arcgis/rest/services/vern/mapserver/4/query?where=1%3D1&outfields=*&f=json&resultRecordCount=1"

**Import data into bevaring_25833.gpkg**

In [8]:
if load_data:
    os.chdir(os.path.join(data_path, "interim"))
    print(os.getcwd())
    
    bioklima_path = catalog["bioclimatic_zones"]["filepath"]
    bioklima_lyr = catalog["bioclimatic_zones"]["layer"]
    ar50_path = catalog["ar50"]["filepath"]
    ar50_lyr = catalog["ar50"]["layer"]

    print(ar50_path, ar50_lyr)
    
    # Copy GDB layers from NINA R to gpkg
    ! ogrinfo "$bioklima_path"
    ! ogr2ogr -f "GPKG" -t_srs EPSG:25833 -nln soner_2017_1km -nlt MULTIPOLYGON bevaring_25833.gpkg "$bioklima_path" "$bioklima_lyr"
    ! ogrinfo "$ar50_path"
    ! ogr2ogr -append -f "GPKG" -t_srs EPSG:25833 -nln "$ar50_lyr" -nlt MULTIPOLYGON bevaring_25833.gpkg "$ar50_path" "$ar50_lyr"


### Raster Data



In [9]:
if load_data:
    infra_path = catalog["infra_25m"]["filepath"]
    
    # copy to project raw folder and convert to 25833
    ! gdalwarp -t_srs EPSG:25833 $infra_path $raw_data/infra_25m_25833.tif

In [10]:
# clip raster by polygon 
if load_data:
    ! gdalwarp -cutline "//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/norge_landareal_n50.shp" -crop_to_cutline \
        $raw_data/infra_25m_25833.tif \
        $interim_data/infra_25m_25833_clip.tif

In [11]:
# WARNING: run in bash shell, otherise correct path to gdal_calc.py is not found. 
# multiply all cells by 1000 except for nodata values. 
if load_data:
    ! gdal_calc.py -A "//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/infra_25m_25833_clip.tif" \
        --outfile="//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/infra_25m_25833_x1000.tif" \
            --calc="numpy.where(A!=3.4e+38, A*1000, 3.4e+38)" --type=Float32 --NoDataValue=3.4e+38

In [12]:
# to integer
if load_data:
    ! gdal_translate -ot Int16 -of GTiff \
        "//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/infra_25m_25833_x1000.tif" \
        "//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/infra_25m_25833_x1000_int.tif"

In [13]:
# clip by coast again 
if load_data:
    ! gdalwarp -cutline "//home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/norge_landareal_n50.shp" -crop_to_cutline \
        $interim_data/infra_25m_25833_x1000_int.tif \
        $interim_data/infra_25m_25833_x1000int_coastalclip.tif 

In [14]:
# raster to vector
# to BIG! 
if load_data:
    ! gdal_polygonize.py $interim_data/infra_25m_25833_x1000_int.tif -f "GeoJSON" $raw_data/infra_25m_25833_x1000_int.geojson

**Write Metadata to .txt**

In [15]:
if load_data: 
    # run shell scripts from /src/shell
    os.chdir(shell_path)

    path = os.path.join(data_path, "interim")

    ! gdalinfo --version
    ! chmod +x gdal_gpkg-info.sh
    ! ./gdal_gpkg-info.sh $path "vern_25833.gpkg"
    ! ./gdal_gpkg-info.sh $path "bevaring_25833.gpkg"
    ! ./gdal_gpkg-info.sh $path "admin_25833.gpkg"

## Data Preparation

### Sudy Area | Norway Mainland

This workflow is created and run for Norway mainland. If you like to test or use the workflow we recommend running it first on a study area. Below are bounding box coordinates provided for Dovrefjell, Fosen or Trondheim area. 

**Create Bounding Box** 

In [16]:
# bounding box Dovrefjell
bounding_box = "fosen"  # "dovrefjell" or "trondheim"

if bounding_box == "dovrefjell":
    xmin, ymin, xmax, ymax = 160000.00, 6900000.00, 260000.00, 6950000.00
if bounding_box == "fosen":
    xmin, ymin, xmax, ymax = 180000.00, 7010000.00, 290000.00, 7150000.00
if bounding_box == "trondheim":
    xmin, ymin, xmax, ymax = 260520.12, 7032142.5, 278587.56, 7045245.27

# Create a bounding box
boxBB =  box(xmin, ymin, xmax, ymax)
crs = 'EPSG:25833'

gdf_BB = gpd.GeoDataFrame(geometry=[boxBB])
gdf_BB['name'] = 'Dovre_BB'
gdf_BB.crs = crs
bounds = gdf_BB.bounds.to_numpy().tolist()[0]

In [17]:
if load_data: 
    # run shell scripts from /src/shell
    os.chdir(shell_path)
    for file in ["vern", "bevaring", "admin"]:
        input_file = os.path.join(data_path, "interim", f"{file}_25833.gpkg")
        output_file = os.path.join(data_path, "tmp", f"{file}_25833_bbox.gpkg")
        
        #for layer_name in layer_names:
        ! chmod +x gdal_copy-file-bbox.sh
        ! ./gdal_copy-file-bbox.sh {input_file} {output_file} {xmin} {ymin} {xmax} {ymax}

In [18]:
if load_data: 
    # create a temporary test gpkg
    tmp_gpkg = os.path.join(data_path, "tmp", "tmp.gpkg")
    in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
    print(tmp_gpkg)

    sql_query = f'SELECT * FROM "ar50_flate" LIMIT 5'
    path = os.path.join(data_path, "tmp")
    ! ogr2ogr -f "GPKG" -nln "ar50_flate" -sql "$sql_query" {tmp_gpkg} {in_gpkg}

### AR50 - Bonitet

Translate the Bonitet classes using a lookup table.
 
&emsp; $Bonitet = (artype, artreslag, arskogbon, arjordbr, ardyrkning, arveget)$ 

In [19]:
# ar50 layer
layer_name, new_field_name = "ar50_flate", "ar50_bonitet"

if test_area:
    # import gpkg into ogr object 
    # add field name if it does not exist
    tmp_gpkg = os.path.join(data_path, "tmp", "bevaring_25833_bbox.gpkg")
    ds, lyr = import_gpkg(tmp_gpkg, layer_name, new_field_name)
    #print_layer_schema(lyr)
else:
    # import gpkg into ogr object 
    # add field name if it does not exist
    in_gpkg = os.path.join(interim_data, "bevaring_25833.gpkg")
    print(in_gpkg)
    ds, lyr = import_gpkg(in_gpkg, layer_name, new_field_name)
    #print_layer_schema(lyr)

# Convert the first 5 features of the layer to a DataFrame
# use islice to limit the number of features to 5, to reduce computation time
df_AR50 = pd.DataFrame(feature.items() for feature in islice(lyr, 5))

# Print the DataFrame
df_AR50

/home/NINA.NO/willeke.acampo/Mounts/scratch/wilaca/vern_og_bevaring/data/interim/bevaring_25833.gpkg


,objtype,artype,arskogbon,artreslag,arveget,arjordbr,ardyrking,arkartstd,målemetode,informasjon,oppdateringsdato,originaldatavert,navnerom,kopidato,områdeid,lokalid,fylkeid,komid,ar50_bonitet
0,arealressursflate,60,13,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,1,50,5022,None
1,arealressursflate,60,11,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,2,50,5022,None
2,arealressursflate,60,13,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,3,50,5022,None
3,arealressursflate,60,11,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,4,50,5022,None
4,arealressursflate,30,13,31,98,98,81,AR50,64,Generalisert fra AR5 Årsversjon 2016,20180106,NIBIO,NO_NIBIO_AR50_2019_2,20211021,5022,5,50,5022,None


In [20]:
# lookup table 
lookup_csv = os.path.join(data_path, "AR50_bonitet_lookup.csv")
lookup_df = pd.read_csv(lookup_csv)

# rename cols to correspond with AR50 lyr
lookup_df.rename(
    columns={
        "ARTYPE kode": "artype",
        "ARTRESLAG kode": "artreslag",
        "ARSKOGBON kode": "arskogbon",
        "ARJORDBR kode": "arjordbr",
        "ARDYRKING kode": "ardyrking",
        "ARVEGET kode": "arveget",
        "Bonitet kode": "ar50_bonitet",
    },
    inplace=True,
)

# reorder cols to correspond with AR50 lyr
lookup_df = lookup_df[
    [
        "artype",
        "arskogbon",
        "artreslag",
        "arveget",
        "arjordbr",
        "ardyrking",
        "Beskrivelse",
        "ar50_bonitet",
    ]
]

display(lookup_df.head(11))

,artype,arskogbon,artreslag,arveget,arjordbr,ardyrking,Beskrivelse,ar50_bonitet
0,10,98,98,98,98,98,Bebygd og samferdsel,14
1,20,98,98,98,24,98,Fulldyrka og Overflatedyrka jord,1
2,20,98,98,98,25,82,"Innmarksbeite, dyrkbar jord",2
3,20,98,98,98,25,81,Innmarksbeite,2
4,20,98,98,98,99,99,"Jordbruksareal, uspesifisert",1
5,30,11,31,98,98,82,"Barskog, impediment, dyrkbar jord",6
6,30,11,31,98,98,81,"Barskog, impediment",6
7,30,12,31,98,98,82,"Barskog, lav skogbonitet, dyrkbar jord",5
8,30,12,31,98,98,81,"Barskog, lav skogbonitet",5
9,30,13,31,98,98,82,"Barskog, middels skogbonitet, dyrkbar jord",4


In [21]:
# create lookup dict
# keys must be in same order as gpkg lyr fields
keys = (
    "artype",
    "arskogbon",
    "artreslag",
    "arveget",
    "arjordbr",
    "ardyrking",
    )

value= "ar50_bonitet"

lookup_dict = create_lookup_dict(
    lookup_df,
    keys=keys,
    value=value
)

# print first two entries of dict
print({k: lookup_dict[k] for k in list(lookup_dict)[:11]})

{(10, 98, 98, 98, 98, 98): 14, (20, 98, 98, 98, 24, 98): 1, (20, 98, 98, 98, 25, 82): 2, (20, 98, 98, 98, 25, 81): 2, (20, 98, 98, 98, 99, 99): 1, (30, 11, 31, 98, 98, 82): 6, (30, 11, 31, 98, 98, 81): 6, (30, 12, 31, 98, 98, 82): 5, (30, 12, 31, 98, 98, 81): 5, (30, 13, 31, 98, 98, 82): 4, (30, 13, 31, 98, 98, 81): 4}


In [22]:
if prepare_data:
    # loop through the features and reclassify the attribute value "ar50_bonitet"
    features_to_update = []

    for feature in lyr:
        # get the attribute values
        artype = feature.GetField("artype")
        arskogbon = feature.GetField("arskogbon")
        artreslag = feature.GetField("artreslag")
        arveget = feature.GetField("arveget")
        arjordbr = feature.GetField("arjordbr")
        ardyrking = feature.GetField("ardyrking")

        key = (int(artype), int(arskogbon), int(artreslag), int(arveget), int(arjordbr), int(ardyrking))
        if key in lookup_dict:
            new_value = lookup_dict[key]
            feature.SetField("ar50_bonitet", new_value)
            features_to_update.append(feature)

    print(f"Number of features to update: {len(features_to_update)}")
    # Batch update features
    for feature in features_to_update:
        lyr.SetFeature(feature)

    print("Finished updating Bonitet.")

    df_AR50_bon = pd.DataFrame(feature.items() for feature in islice(lyr, 5))
    display(df_AR50_bon.head())

# close OGR object
ds = None

In [23]:
if prepare_data:
    # Define the SQL queries
    sql_marine = f"SELECT * FROM ar50_flate WHERE ar50_bonitet = 17"
    sql_terrestisk = f"SELECT * FROM ar50_flate WHERE ar50_bonitet != 17"

    if test_area:
        # import gpkg into ogr object 
        # add field name if it does not exist
        in_gpkg = os.path.join(data_path, "tmp", "bevaring_25833_bbox.gpkg")
        out_gpkg = os.path.join(data_path, "tmp", "interim_25833_bbox.gpkg")
    else:
        # import gpkg into ogr object 
        # add field name if it does not exist
        in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
        out_gpkg = os.path.join(data_path, "interim", "interim_25833.gpkg")

    print(out_gpkg)

    # Create the ar50_marine layer
    ! ogr2ogr -f "GPKG" -nln "ar50_marine" -sql "$sql_marine" -append {out_gpkg} {in_gpkg}
    # Create the ar50_terrestisk layer
    ! ogr2ogr -f "GPKG" -nln "ar50_terrestisk" -sql "$sql_terrestisk" -append {out_gpkg} {in_gpkg}

### Infrastructure index

- Calculate and display the distribution of the infrastructure index for the whole of norway 
- Calculate and display the distribution of the infrastructure index per region
- Calculate and display the distribution of the infrastructure index per protected area
- Define the infrastructure index classes

### Topographic height

Classify the topographic height into 4 classes:
- 0-300m
- 301-600m
- 601-900m
- over 900m

## Load data into Spatial Database (DuckDB)


### Load data into DuckDB

In [24]:
if test_area:
    db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
else:
    db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")

**Load Bioklima soner into DuckDB**

In [25]:
if prepare_duckdb:
    # AR50 into duckdb
    if test_area:
        in_gpkg = os.path.join(data_path, "tmp", "bevaring_25833_bbox.gpkg")
        db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
        load_gpkg_layers(db_path, in_gpkg, "soner_2017_1km")
    else:
        in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
        db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
        load_gpkg_layers(db_path, in_gpkg, "soner_2017_1km")

**Load AR50 into DuckDB**

In [26]:
# run time: 
if prepare_duckdb:
    # AR50 into duckdb
    if test_area:
        in_gpkg = os.path.join(data_path, "tmp", "bevaring_25833_bbox.gpkg")
        db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
        load_gpkg_layers(db_path, in_gpkg, "ar50_flate")
        load_gpkg_layers(db_path, in_gpkg, "ar50_flate_msk")
    else:
        in_gpkg = os.path.join(data_path, "interim", "bevaring_25833.gpkg")
        db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
        load_gpkg_layers(db_path, in_gpkg, "ar50_flate")

**Load Protected Areas into DuckDB**

In [27]:
# import protected areas into DuckDB

# if Test is True, import test gpkg otherwise import the original gpkg
if prepare_duckdb:
    if test_area:
        in_gpkg = os.path.join(data_path, "tmp", "vern_25833_bbox.gpkg")
        db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
        load_gpkg_layers(db_path, in_gpkg, "naturvernomrade")
        load_gpkg_layers(db_path, in_gpkg, "foreslatt_vern")
    else:
        in_gpkg = os.path.join(data_path, "interim", "vern_25833.gpkg")
        db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
        load_gpkg_layers(db_path, in_gpkg, "naturvernomrade")
        load_gpkg_layers(db_path, in_gpkg, "foreslatt_vern")

**Load Regions into DuckDB**

In [28]:
if prepare_duckdb:
    if test_area:
        in_gpkg = os.path.join(data_path, "tmp", "admin_25833.gpkg")
        db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
        load_gpkg_layers(db_path, in_gpkg, "region_2024")
        load_gpkg_layers(db_path, in_gpkg, "norge_landareal_n50")
    else:
        in_gpkg = os.path.join(data_path, "interim", "admin_25833.gpkg")
        db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
        load_gpkg_layers(db_path, in_gpkg, "region_2024")
        load_gpkg_layers(db_path, in_gpkg, "norge_landareal_n50")

**Load SSB 10km Grid into DuckDB**

In [29]:
# import SSB 10km grid

if prepare_duckdb:
    if test_area:
        in_gpkg = os.path.join(data_path, "tmp", "admin_25833.gpkg")
        db_path = os.path.join(data_path, "tmp", "verg_og_bevaring_tmp.db")
        load_gpkg_layers(db_path, in_gpkg, "norge_ruter_10km")
    else:
        in_gpkg = os.path.join(data_path, "interim", "admin_25833.gpkg")
        db_path = os.path.join(data_path, "interim", "verg_og_bevaring.db")
        load_gpkg_layers(db_path, in_gpkg, "norge_ruter_10km")

### Preprocess DuckDB Tables 

**Convert BLOB columns to GEOM columns**

In [30]:
if prepare_duckdb:
    # cast BLOB (Binary Large OBject) to geometry for spatial operations
    
    import duckdb

    con = duckdb.connect(db_path)
    con.install_extension('spatial')
    con.load_extension('spatial')
    
    # duckdb tables names to list
    tables = con.execute("SHOW TABLES;").fetchdf()
    tables = tables["name"].to_list()
    print(tables)
    
    for table in tables:
        
        blob_to_geom(
            db_path=db_path,
            tbl_name=table,
            blob_field="geometry",
            geom_field="geom",   
        )

### Display DuckDB tables 

In [31]:
import duckdb

con = duckdb.connect(db_path)
con.install_extension('spatial')
con.load_extension('spatial')

In [32]:
reset_method =True 

tables_to_keep = [
    "ar50_flate", 
    "naturvernomrade", 
    "foreslatt_vern", 
    "region_2024", 
    "norge_landareal_n50",
    "norge_ruter_10km", 
    "soner_2017_1km"
    ]

all_tables = [item[0] for item in con.execute("SHOW TABLES;").fetchall()]
print(all_tables)

if reset_method:
    # Iterate over all tables and remove tables that are not in the list of tables to keep
    [remove_table(db_path, tbl) for tbl in all_tables if tbl not in tables_to_keep]

# print tables
con.sql("SHOW TABLES;")

['ar50_flate', 'foreslatt_vern', 'norge_landareal_n50', 'norge_ruter_10km', 'region_2024', 'soner_2017_1km']


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ ar50_flate          │
│ foreslatt_vern      │
│ norge_landareal_n50 │
│ norge_ruter_10km    │
│ region_2024         │
│ soner_2017_1km      │
└─────────────────────┘

In [33]:
# display first 5 rows of the table
table = "foreslatt_vern"
con.sql(f"SELECT * FROM {table} LIMIT 5")

┌──────────────────────┬──────────────────────┬────────────┬───┬───────────────┬───────────┬──────────────────────┐
│ identifikasjon_lok…  │ identifikasjon_nav…  │    navn    │ … │   verneform   │ verneplan │         geom         │
│       varchar        │       varchar        │  varchar   │   │    varchar    │  varchar  │       geometry       │
├──────────────────────┼──────────────────────┼────────────┼───┼───────────────┼───────────┼──────────────────────┤
│ VP00000313           │ http://www.miljodi…  │ Inngripan  │ … │ naturreservat │           │ MULTIPOLYGON (((13…  │
│ VP00000404           │ http://www.miljodi…  │ Kverndalen │ … │ naturreservat │           │ MULTIPOLYGON (((34…  │
│ VP00000410           │ http://www.miljodi…  │ Båsdalen   │ … │ naturreservat │           │ MULTIPOLYGON (((41…  │
│ VP00000416           │ http://www.miljodi…  │ Rennen     │ … │ naturreservat │           │ MULTIPOLYGON (((29…  │
│ VP00000418           │ http://www.miljodi…  │ Leksa      │ … │ naturre

In [34]:
# close
con.close()

## Display Data on the Map

https://ipyleaflet.readthedocs.io/en/latest/installation/index.html#using-pip 


In [35]:
# Display data on map using leafmap
# TODO display duckdb tables as well, not only WMS

init_location = [62.223207, 9.550195]  # Hjerkinn
zoom_start = 13  
min_zoom = 8  # keeps user from zooming out too far
basemap = leafmap.Map(
    location=init_location, 
    zoom=zoom_start, 
    min_zoom=min_zoom, 
    max_bounds=True
    )

# set background
basemap.add_basemap("SATELLITE", opacity=0.7)

# add WMS verneområder
wms_url ="https://kart.miljodirektoratet.no/arcgis/services/vern/mapserver/WMSServer"
wms_layer = "naturvern_klasser_omrade"
wms_name = "Protected Areas"
basemap.add_wms_layer(
    url=wms_url, 
    layers=wms_layer, 
    name=wms_name, 
    wms_format="image/png",
    )

if test_area:
    # add bounding box - Hjerkinn
    basemap.add_gdf(gdf_BB, layer_name="Test Area", fill_color="blue", fill_opacity=0.2, weight=2)

basemap

# Methods

In [36]:
if statistical_zone == "norge_landareal_n50":
    # add a column "identifikasjon_lokalId" to the table "norge_landareal_n50"
    # fill column 0 - x
    con = duckdb.connect(db_path)
    con.install_extension('spatial')
    con.load_extension('spatial')
    
    # merge all rows into one multipolygon
    #con.execute("""
    #    CREATE TABLE norge_landareal_n50_union AS
    #    SELECT ST_Union_Agg(geom) AS geom
    #    FROM norge_landareal_n50
    #""")
    
    
    con.execute("ALTER TABLE norge_landareal_n50 ADD COLUMN identifikasjon_lokalId INTEGER;")
    con.execute("UPDATE norge_landareal_n50 SET identifikasjon_lokalId = rowid;")
    con.close()
    
    statistical_zone = "norge_landareal_n50_union"

**Infrastrukturindeks**

| Protected area | Mean value | Lav (%) | Middels (%) | Høy (%) | ... |
|----------------|------------|---------|-------------|---------|-----|
| area A         |            |         |             |         |     |


In [37]:
# table to df 

# to dataframe
con = duckdb.connect(db_path)
con.install_extension('spatial')
con.load_extension('spatial')

df = con.execute(f"SELECT ST_AsText(geom) as geometry, * FROM {statistical_zone}").fetchdf()

# drop "geom"
df = df.drop(columns=["geom"])

# Convert the geom column to gpd GeoDataFrame
df['geometry'] = df['geometry'].apply(wkt.loads)

# to gpd
gdf = gpd.GeoDataFrame(df, geometry='geometry')
print(f"Number of unique ids: {gdf['identifikasjon_lokalId'].nunique()}")
print(f"Number of rows: {gdf.shape[0]}")

# drop all rows where id is NULL
gdf = gdf.dropna(subset=["identifikasjon_lokalId"])

display(gdf.head())
con.close()

Number of unique ids: 289
Number of rows: 289


,geometry,identifikasjon_lokalId,identifikasjon_navnerom,navn,datafangstdato,faktaark,kommune,kvalitet_målemetode,kvalitet_nøyaktighet,verneform,verneplan
0,"MULTIPOLYGON (((130580.250 7026964.960, 130698...",VP00000313,http://www.miljodirektoratet.no/naturbase,Inngripan,20031217000000,https://faktaark.naturbase.no/?id=VP00000313,1505,NaN,NaN,naturreservat,
1,"MULTIPOLYGON (((346384.230 7072068.630, 346949...",VP00000404,http://www.miljodirektoratet.no/naturbase,Kverndalen,20000101000000,https://faktaark.naturbase.no/?id=VP00000404,5038,NaN,NaN,naturreservat,
2,"MULTIPOLYGON (((413006.410 7154107.450, 413032...",VP00000410,http://www.miljodirektoratet.no/naturbase,Båsdalen,20000101000000,https://faktaark.naturbase.no/?id=VP00000410,5042,NaN,NaN,naturreservat,
3,"MULTIPOLYGON (((297926.190 7033095.820, 297935...",VP00000416,http://www.miljodirektoratet.no/naturbase,Rennen,20000101000000,https://faktaark.naturbase.no/?id=VP00000416,5035,NaN,NaN,naturreservat,
4,"MULTIPOLYGON (((306858.760 7029574.680, 306864...",VP00000418,http://www.miljodirektoratet.no/naturbase,Leksa,20000101000000,https://faktaark.naturbase.no/?id=VP00000418,5035,NaN,NaN,naturreservat,


In [38]:
if run_infra:
    from py_scripts.raster import * 

    # calculate the min, max, mean, std and median of the infrastruktur raster 
    # within the protected areas with a buffer of 1000m
    raster_path = os.path.join(interim_data, "infra_25m_25833_clip.tif")

    if statistical_zone == "norge_landareal_n50":
        buffer_distance = 0
    else:
        buffer_distance = 1000
    print(buffer_distance)
    
    gdf_infra = overlay_stats(raster_path, gdf, buffer_distance=buffer_distance)
    
    display(gdf_infra.head())

1000


,geometry,identifikasjon_lokalId,identifikasjon_navnerom,navn,datafangstdato,faktaark,kommune,kvalitet_målemetode,kvalitet_nøyaktighet,verneform,verneplan,infra_min,infra_max,infra_mean,infra_std,infra_median
0,"MULTIPOLYGON (((130580.250 7026964.960, 130698...",VP00000313,http://www.miljodirektoratet.no/naturbase,Inngripan,20031217000000,https://faktaark.naturbase.no/?id=VP00000313,1505,NaN,NaN,naturreservat,,0.0,0.643856,0.078382,0.210531,0.0
1,"MULTIPOLYGON (((346384.230 7072068.630, 346949...",VP00000404,http://www.miljodirektoratet.no/naturbase,Kverndalen,20000101000000,https://faktaark.naturbase.no/?id=VP00000404,5038,NaN,NaN,naturreservat,,0.0,2.000000,0.017769,0.156851,0.0
2,"MULTIPOLYGON (((413006.410 7154107.450, 413032...",VP00000410,http://www.miljodirektoratet.no/naturbase,Båsdalen,20000101000000,https://faktaark.naturbase.no/?id=VP00000410,5042,NaN,NaN,naturreservat,,0.0,2.087463,0.066969,0.321397,0.0
3,"MULTIPOLYGON (((297926.190 7033095.820, 297935...",VP00000416,http://www.miljodirektoratet.no/naturbase,Rennen,20000101000000,https://faktaark.naturbase.no/?id=VP00000416,5035,NaN,NaN,naturreservat,,0.0,4.983706,0.295746,0.742960,0.0
4,"MULTIPOLYGON (((306858.760 7029574.680, 306864...",VP00000418,http://www.miljodirektoratet.no/naturbase,Leksa,20000101000000,https://faktaark.naturbase.no/?id=VP00000418,5035,NaN,NaN,naturreservat,,0.0,5.422065,0.169378,0.572697,0.0


In [39]:
# remove table study area
remove_table(db_path, str(statistical_zone))

In [40]:
con = duckdb.connect(db_path)
# print tables
con.sql("SHOW TABLES;")

┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ ar50_flate          │
│ norge_landareal_n50 │
│ norge_ruter_10km    │
│ region_2024         │
│ soner_2017_1km      │
└─────────────────────┘

In [41]:
con.close()

In [42]:
# export to gpkg interim
out_gpkg = os.path.join(interim_data, "interim_25833.gpkg")
out_layer = f"{statistical_zone}_infra"
# export to specifc layer "stastiscial_zone_infra"

gdf_infra.crs = "EPSG:25833"
# Write to existing .geopackage
gdf_infra.to_file(out_gpkg, driver='GPKG', layer=out_layer, mode='w')

In [45]:
# to duckdb
load_gpkg_layers(db_path, out_gpkg, out_layer)

# convert geom
blob_to_geom(
    db_path=db_path,
    tbl_name=out_layer,
    blob_field="geometry",
    geom_field="geom",   
)


Table foreslatt_vern_infra already exists in DuckDB database. Skipping.
An error occurred: Binder Error: Referenced column "geometry" not found in FROM clause!
Candidate bindings: "foreslatt_vern_infra.geom"
LINE 3: ...               CREATE TABLE foreslatt_vern_infra_tmp AS 
                SELECT *, ST_GeomFromWKB(geometry) AS geom
                                                  ^



### 1. Overlap protected areas and Bioclimatic Zones
**Bioklimatiske soner**

| Protected area | Sone 1 (%) | Sone 2 (%) | Sone 3 (%) | ... |
|----------------|------------|------------|------------|-----|
| *NaturvernId*  |*bioklima_1*|*bioklima_2*|*bioklima_3*|...  |
| area A         |            |            |            |     |

In [46]:
if run_bioklima:
    tbl_study_area = f"{statistical_zone}_infra"
    id = "identifikasjon_lokalId"
    tbl_bioklima = "soner_2017_1km"
    bioklima_field = "Sone_Kode"
    bioklima_zones = ["6SO-1", "6SO-2", "6SO-3", "6SO-4", "6SO-5"]
    for area_class in bioklima_zones: # 1-18
        print(f"Calculating Bioklima class {area_class}")
        new_field = f"s_{area_class}_m2" # ar50_bon1
        new_field = new_field.replace("-", "_")
        remove_field(db_path, tbl_study_area, new_field)
        bioklima_area_class(db_path, tbl_study_area, id, tbl_bioklima, bioklima_field, area_class, new_field)
    
    for area_class in bioklima_zones: # 1-18
        new_field = f"s_{area_class}_m2" # ar50_bon1
        new_field = new_field.replace("-", "_")
        con = duckdb.connect(db_path)
        con.install_extension('spatial')
        con.load_extension('spatial')
        con.sql(f"UPDATE {tbl_study_area} SET {new_field} = 0 WHERE {new_field} IS NULL")
        con.close()

Calculating Bioklima class 6SO-1


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Calculating Bioklima class 6SO-2


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Calculating Bioklima class 6SO-3


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Calculating Bioklima class 6SO-4


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Calculating Bioklima class 6SO-5


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [47]:
con = duckdb.connect(db_path)
con.sql(f"SELECT * FROM {tbl_study_area} LIMIT 5")

┌──────────────────────┬──────────────────────┬────────────┬───┬────────────┬────────────┬────────────┬────────────┐
│ identifikasjon_lok…  │ identifikasjon_nav…  │    navn    │ … │ s_6SO_2_m2 │ s_6SO_3_m2 │ s_6SO_4_m2 │ s_6SO_5_m2 │
│       varchar        │       varchar        │  varchar   │   │   float    │   float    │   float    │   float    │
├──────────────────────┼──────────────────────┼────────────┼───┼────────────┼────────────┼────────────┼────────────┤
│ VP00000313           │ http://www.miljodi…  │ Inngripan  │ … │        0.0 │        0.0 │        0.0 │        0.0 │
│ VP00000404           │ http://www.miljodi…  │ Kverndalen │ … │        0.0 │  5992017.5 │  5007369.5 │        0.0 │
│ VP00000410           │ http://www.miljodi…  │ Båsdalen   │ … │        0.0 │   536979.1 │  1080859.9 │        0.0 │
│ VP00000416           │ http://www.miljodi…  │ Rennen     │ … │        0.0 │  1811234.5 │        0.0 │        0.0 │
│ VP00000418           │ http://www.miljodi…  │ Leksa      │ … │

### 2. Overlap protected areas and AR50 (land cover)

**Divide the protected areas into terrestrial and marine areas**

AR50 polygons with classified as "Hav" are considered marine areas, all other areas are considered terrestrial.
- Bonitet = 17 (Hav) 
- Arealtype = 82 (Hav)

<br>

**AR50 - Bonitet**
Calculated area overlap with AR50 Bonitet classes. 

| Protected area | Fulldyrka og overflatedyrka jord (m2) | Innmarksbeite (m2) | Skog, høg og særs høg bonitet (m2) | ... |
|----------------|--------------------------------------|-------------------|-----------------------------------|-----|
| *NaturvernId*  |*AR50_bon_1*|*AR50_bon2*|*AR50_bon3*|...|
| area A         |                                      |                   |                                   |     |


In [48]:
# calculate ar50 area overlapp with protected areas
tbl_study_area = f"{statistical_zone}_infra"
id = "identifikasjon_lokalId"
tbl_ar50 = "ar50_flate"
ar50_field = "ar50_bonitet"

if run_ar50:
    for area_class in range(1, 19): # 1-18
        print(f"Calculating Bonitet class {area_class}")
        new_field = f"ar50_bon{area_class}_m2" # ar50_bon1
        remove_field(db_path, tbl_study_area, new_field)
        ar50_area_class(db_path, tbl_study_area, id, tbl_ar50, ar50_field, area_class, new_field)

    # if ar50_bonx is null, set to 0
    for area_class in range(1, 19): # 1-18
        new_field = f"ar50_bon{area_class}_m2" # ar50_bon1
        con = duckdb.connect(db_path)
        con.install_extension('spatial')
        con.load_extension('spatial')
        con.sql(f"UPDATE {tbl_study_area} SET {new_field} = 0 WHERE {new_field} IS NULL")
        con.close()

Calculating Bonitet class 1


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Calculating Bonitet class 2


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Calculating Bonitet class 3


...

Calculating Bonitet class 13


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Calculating Bonitet class 14
Calculating Bonitet class 15
Calculating Bonitet class 16


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Calculating Bonitet class 17
Calculating Bonitet class 18


In [49]:
# print first 5 rows of the table statistical_zone
con = duckdb.connect(db_path)
con.sql(f"SELECT * FROM {tbl_study_area} LIMIT 5")

# print as dataframe
df = con.execute(f"SELECT * FROM {tbl_study_area} LIMIT 5").fetchdf()

# display columns: identfifikasjon_lokalId, ar50_bonx
col_id= [id]
cols_ar50 = [col for col in df.columns if "ar50_bon" in col and col != "sum_ar50_bon_m2"]
cols = col_id + cols_ar50
df_ar50 = df[cols]
display(df)

# close
con.close()

,identifikasjon_lokalId,identifikasjon_navnerom,navn,datafangstdato,faktaark,kommune,kvalitet_målemetode,kvalitet_nøyaktighet,verneform,verneplan,...,ar50_bon9_m2,ar50_bon10_m2,ar50_bon11_m2,ar50_bon12_m2,ar50_bon13_m2,ar50_bon14_m2,ar50_bon15_m2,ar50_bon16_m2,ar50_bon17_m2,ar50_bon18_m2
0,VP00000313,http://www.miljodirektoratet.no/naturbase,Inngripan,20031217000000,https://faktaark.naturbase.no/?id=VP00000313,1505,NaN,NaN,naturreservat,,...,0.000000e+00,32083.675781,0.000000,0.000000,0.0,0.0,0.0,0.000000,4117838.75,0.0
1,VP00000404,http://www.miljodirektoratet.no/naturbase,Kverndalen,20000101000000,https://faktaark.naturbase.no/?id=VP00000404,5038,NaN,NaN,naturreservat,,...,2.108428e+06,128523.148438,188387.609375,225231.843750,0.0,0.0,0.0,41450.535156,0.00,0.0
2,VP00000410,http://www.miljodirektoratet.no/naturbase,Båsdalen,20000101000000,https://faktaark.naturbase.no/?id=VP00000410,5042,NaN,NaN,naturreservat,,...,1.802707e+04,0.000000,69037.406250,40107.097656,0.0,0.0,0.0,32741.802734,0.00,0.0
3,VP00000416,http://www.miljodirektoratet.no/naturbase,Rennen,20000101000000,https://faktaark.naturbase.no/?id=VP00000416,5035,NaN,NaN,naturreservat,,...,0.000000e+00,0.000000,55577.246094,28114.167969,0.0,0.0,0.0,485110.375000,0.00,0.0
4,VP00000418,http://www.miljodirektoratet.no/naturbase,Leksa,20000101000000,https://faktaark.naturbase.no/?id=VP00000418,5035,NaN,NaN,naturreservat,,...,1.806216e+05,0.000000,627368.187500,498110.281250,0.0,0.0,0.0,0.000000,0.00,0.0


**Check if sum of all AR50 classes correspons with the total area**

In [50]:
# Calculating sum of all bonitet classes
remove_field(db_path, tbl_study_area, "sum_ar50_bon_m2")
sum_area_cols(db_path, tbl_study_area, id, cols_ar50, "sum_ar50_bon_m2")

# Calculating area of each polygon in the study area
remove_field(db_path, tbl_study_area, "areal_m2")
geom_area(db_path, tbl_study_area, "geom", "areal_m2")

# Calculateing the difference between the sum of all bonitet classes and the area of each polygon
remove_field(db_path, tbl_study_area, "area_diff_m2")
area_difference(db_path, tbl_study_area, "sum_ar50_bon_m2", "areal_m2", "area_diff_m2")

In [51]:
# print first 5 rows of the table statistical_zone
con = duckdb.connect(db_path)

# print as dataframe
df = con.execute(f"SELECT * FROM {tbl_study_area}").fetchdf()

# display columns: identfifikasjon_lokalId, ar50_bonx
col_id= [id]
cols = col_id + ["navn", "sum_ar50_bon_m2", "areal_m2", "area_diff_m2"]
df_ar50 = df[cols]

# sort by area_diff_m2
df_ar50 = df_ar50.sort_values(by="area_diff_m2", ascending=True)
display(df_ar50)

# close
con.close()

,identifikasjon_lokalId,navn,sum_ar50_bon_m2,areal_m2,area_diff_m2
62,VP00001251,Nedre Adventdalen,0.000000e+00,8.532030e+07,-85320304.0
117,VP00000024,Goahteluoppal,3.257684e+08,3.258326e+08,-64160.0
165,VP00000450,Anárjohka naturreservat,2.357349e+07,2.360664e+07,-33152.0
172,VP00000682,Andøyatransektet,1.964836e+09,1.964847e+09,-11776.0
167,VP00000452,Anárjohka nasjonalpark,4.752241e+08,4.752264e+08,-2304.0
...,...,...,...,...,...
5,VP00000423,Fjelløya,4.528830e+06,4.528830e+06,0.5
32,VP00001052,Homvassdalen NR utvidelse,1.581100e+07,1.581100e+07,1.0
1,VP00000404,Kverndalen,1.099939e+07,1.099939e+07,1.0
161,VP00000442,Merralia,1.407740e+07,1.407740e+07,1.0


In [52]:
# print number of rows 
print(f"Number of rows: {df.shape[0]}")

# print number of unique ids "identifikasjon_lokalId"
print(f"Number of unique ids: {df['identifikasjon_lokalId'].nunique()}")

# print number of rows with value in "ar50_bon17"
print(f"Number of areas with Marine area: {df['ar50_bon17_m2'].count()}")

Number of rows: 289
Number of unique ids: 289
Number of areas with Marine area: 289


### 2. Geometry variables for protected areas

| Protected area | Total area (km2) | Perimeter (km) | Land area (km2) | Perimeter land area (km) |
|----------------|------------|----------------|-----------------|-------------------------|
| *NaturvernId* | *areal_m2* | *omkrets_m* | *landareal_m2* | *landomkrets_m2*|
| area A         |    *        |                |                 |                         |

***
$\mathbf{\text{Shape Index}}$<br>
***
The shape index is a measure of how compact the shape is compared to a circle with the same area. The shape index is calculated for the entire protected area and the land area.

- $P$ = perimeter
- $r$ = radius
- $A$ = area


**Shape Index:**&emsp;      $SI = \frac{P}{2\pi r}$

**Radius:**&emsp; $r = \sqrt{\frac{A}{\pi}}$

- SI = 1, shape is a perfect circle
- SI > 1, shape is less compact than a circle
- SI < 1, is not possible 

In [53]:
if statistical_zone == "norge_landareal_n50":
    
    
    # to dataframe
    df = con.execute(f"SELECT ST_AsText(geom) as geometry, * FROM {tbl_study_area}").fetchdf()

    # drop "geom"
    df = df.drop(columns=["geom"])

    # Convert the geom column to gpd GeoDataFrame
    df['geometry'] = df['geometry'].apply(wkt.loads)

    # to gpd
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    print(f"Number of unique ids: {gdf['identifikasjon_lokalId'].nunique()}")


    display(gdf)
        
    # quite notebook
    # Export GDF to file 
    filepath = os.path.join(data_path, "processed", "vern_og_bevaring")
    gdf.crs = "EPSG:25833"
    # Write to existing .geopackage
    gdf.to_file(os.path.join(filepath + '.gpkg'), driver='GPKG', layer=f'{statistical_zone}_overlapp', mode='w')

    # gdf to df and remove geom
    df = gdf.drop(columns=["geometry"])
    # Write to .csv
    df.to_csv(os.path.join(data_path, "processed", f'{statistical_zone}_overlapp.csv'))
    quit()

In [54]:
if run_shape_index:
    # create two new tables with only terrestrial part and marine part, respectively.
    remove_table(db_path, sz_land)
    remove_table(db_path, sz_sea)
    remove_table(db_path, f"{sz_land}_tmp1")
    remove_table(db_path, f"{sz_sea}_tmp1")
    extract_overlap_geom(db_path, id, tbl_ar50, "ar50_bonitet", 17, tbl_study_area, f"{sz_sea}_tmp1", f"{sz_land}_tmp1")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [55]:
if run_shape_index:

    # remove slivers (lines/points) from the tables
    remove_table(db_path, f"{sz_land}_tmp2")
    remove_table(db_path, f"{sz_sea}_tmp2")
    delete_lines_points(db_path, f"{sz_land}_tmp1", f"{sz_land}_tmp2")
    delete_lines_points(db_path, f"{sz_sea}_tmp1", f"{sz_sea}_tmp2")

    # to multipolygon
    remove_table(db_path, sz_land)
    remove_table(db_path, sz_sea)
    group_to_multipolygon(db_path, f"{sz_land}_tmp2", sz_land, id)
    group_to_multipolygon(db_path, f"{sz_sea}_tmp2", sz_sea, id)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [56]:
dict_geom = {
    tbl_study_area: ("areal_m2", "omkrets_m", "formindeks"),
    sz_land: ("landareal_m2", "landomkrets_m", "land_formindeks"),
    sz_sea: ("sjoareal_m2", "sjoomkrets_m", "sjo_formindeks"),
}

for key, value in dict_geom.items():
    
    # areal
    #remove_field(db_path, key, "areal_not_grouped")
    #geom_area(db_path, key, "geom", "areal_not_grouped")
    
    # areal by ID
    remove_field(db_path, key, value[0])
    geom_area_byID(db_path, key, "identifikasjon_lokalId", "geom", value[0])
    
    # omkrets
    #remove_field(db_path, key, "omkrets_not_grouped")
    #geom_peri(db_path, key, "geom", "omkrets_not_grouped")
    
    # omkrets by ID
    remove_field(db_path, key, value[1])
    geom_peri_byID(db_path, key, "identifikasjon_lokalId", "geom", value[1])
    
    # formindeks
    #remove_field(db_path, key, "formindeks_not_grouped")
    #geom_index(db_path, key, "geom", "formindeks_not_grouped")
    
    # formindeks by ID
    remove_field(db_path, key, value[2])
    geom_index_byID(db_path, key, "identifikasjon_lokalId", "geom", value[2])


In [57]:
con = duckdb.connect(db_path)
con.install_extension('spatial')
con.load_extension('spatial')

In [58]:
con.sql("SHOW TABLES;")

┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ ar50_flate           │
│ f_vern_land          │
│ f_vern_land_tmp1     │
│ f_vern_land_tmp2     │
│ f_vern_sjo           │
│ f_vern_sjo_tmp1      │
│ f_vern_sjo_tmp2      │
│ foreslatt_vern_infra │
│ norge_landareal_n50  │
│ norge_ruter_10km     │
│ region_2024          │
│ soner_2017_1km       │
├──────────────────────┤
│       12 rows        │
└──────────────────────┘

In [59]:
con.sql(f"SELECT * FROM {sz_land} LIMIT 5")

┌──────────────────────┬──────────────────────────────────────────────┬──────────────┬───────────────┬─────────────────┐
│ identifikasjon_lok…  │                     geom                     │ landareal_m2 │ landomkrets_m │ land_formindeks │
│       varchar        │                   geometry                   │    float     │     float     │      float      │
├──────────────────────┼──────────────────────────────────────────────┼──────────────┼───────────────┼─────────────────┤
│ VP00001350           │ POLYGON ((46174.50999999978 6501172.140000…  │    377724.34 │     3097.9802 │       1.4219562 │
│ VP00001352           │ POLYGON ((95055.0700000003 6468193.9499999…  │    23973.648 │      622.8573 │       1.1347927 │
│ VP00001355           │ POLYGON ((81300.37000000011 6557026.32, 81…  │    377546.22 │     3190.6885 │       1.4648542 │
│ VP00001334           │ POLYGON ((188944.8099999996 6647847.359999…  │   194190.875 │     2150.6907 │       1.3767612 │
│ VP00001336           │ POLYGON

In [60]:
con.sql(f"SELECT * FROM {sz_sea} LIMIT 5")

┌──────────────────────┬─────────────────────────────────────────────────┬─────────────┬──────────────┬────────────────┐
│ identifikasjon_lok…  │                      geom                       │ sjoareal_m2 │ sjoomkrets_m │ sjo_formindeks │
│       varchar        │                    geometry                     │    float    │    float     │     float      │
├──────────────────────┼─────────────────────────────────────────────────┼─────────────┼──────────────┼────────────────┤
│ VP00001322           │ MULTIPOLYGON (((259730.00009999983 6628991.85…  │   1291.6252 │    1323.7974 │      10.390784 │
│ VP00001362           │ MULTIPOLYGON (((254729.96600000001 6601810.02…  │    989.3708 │     550.3659 │       4.935908 │
│ VP00001230           │ POLYGON ((259974.37019999977 6577002.37780000…  │    93092.62 │    1453.3857 │      1.3437495 │
│ VP00001231           │ MULTIPOLYGON (((256981.7921000002 6577341.289…  │  236538.375 │    3956.0566 │      2.2945988 │
│ VP00000556           │ MULTIPO

**Export to GDF and GPKG-layer**

In [61]:
# to dataframe
df_land = con.execute(f"SELECT ST_AsText(geom) as geometry, * FROM {sz_land}").fetchdf()

# drop "geom"
df_land = df_land.drop(columns=["geom"])

# Convert the geom column to gpd GeoDataFrame
df_land['geometry'] = df_land['geometry'].apply(wkt.loads)

# to gpd
gdf_land = gpd.GeoDataFrame(df_land, geometry='geometry')
print(f"Number of unique ids: {df_land['identifikasjon_lokalId'].nunique()}")

display(gdf_land.head())

# Export GDF to file 
filepath = os.path.join(data_path, "processed", "vern_og_bevaring")
gdf_land.crs = "EPSG:25833"
# Write to existing .geopackage
gdf_land.to_file(os.path.join(filepath + '.gpkg'), driver='GPKG', layer=sz_land, mode='w')


Number of unique ids: 287


,geometry,identifikasjon_lokalId,landareal_m2,landomkrets_m,land_formindeks
0,"POLYGON ((46174.510 6501172.140, 46173.960 650...",VP00001350,377724.343750,3097.980225,1.421956
1,"POLYGON ((95055.070 6468193.950, 95055.470 646...",VP00001352,23973.648438,622.857300,1.134793
2,"POLYGON ((81300.370 6557026.320, 81301.610 655...",VP00001355,377546.218750,3190.688477,1.464854
3,"POLYGON ((188944.810 6647847.360, 188951.680 6...",VP00001334,194190.875000,2150.690674,1.376761
4,"POLYGON ((208238.730 6608531.190, 208240.347 6...",VP00001336,150586.140625,2421.995361,1.760661


In [62]:
# to dataframe
df_sjo = con.execute(f"""
    SELECT ST_AsText(geom) as geometry, * 
    FROM {sz_sea}
    WHERE ST_GeometryType(geom) IN ('POLYGON', 'MULTIPOLYGON')
""").fetchdf()

# delete POLYGON EMPTY
df_sjo = df_sjo[df_sjo['geometry'] != "POLYGON EMPTY"]

# drop "geom"
df_sjo = df_sjo.drop(columns=["geom"])

# Convert the geom column to gpd GeoDataFrame
df_sjo['geometry'] = df_sjo['geometry'].apply(wkt.loads)

# to gpd
gdf_sjo = gpd.GeoDataFrame(df_sjo, geometry='geometry')
print(f"Number of unique ids: {gdf_sjo['identifikasjon_lokalId'].nunique()}")

display(gdf_sjo)

# Export GDF to file 
filepath = os.path.join(data_path, "processed", "vern_og_bevaring")
gdf_sjo.crs = "EPSG:25833"
# Write to existing .geopackage
gdf_sjo.to_file(os.path.join(filepath + '.gpkg'), driver='GPKG', layer=sz_sea, mode='w')

Number of unique ids: 43


,geometry,identifikasjon_lokalId,sjoareal_m2,sjoomkrets_m,sjo_formindeks
0,"MULTIPOLYGON (((259730.000 6628991.851, 259731...",VP00001322,1.291625e+03,1.323797e+03,10.390784
1,"MULTIPOLYGON (((254729.966 6601810.029, 254770...",VP00001362,9.893708e+02,5.503659e+02,4.935908
2,"POLYGON ((259974.370 6577002.378, 259984.629 6...",VP00001230,9.309262e+04,1.453386e+03,1.343750
3,"MULTIPOLYGON (((256981.792 6577341.289, 256944...",VP00001231,2.365384e+05,3.956057e+03,2.294599
4,"MULTIPOLYGON (((-51345.730 6717965.518, -51346...",VP00000556,2.083321e+08,2.137686e+05,4.177928
5,"POLYGON ((917639.025 7877543.745, 917594.638 7...",VP00000036,7.858593e+05,4.055916e+03,1.290659
6,"MULTIPOLYGON (((718563.580 7754824.049, 718559...",VP00000352,4.342828e+03,4.338558e+03,18.571808
7,"MULTIPOLYGON (((218505.950 7069450.800, 218507...",VP00000870,6.001958e+06,2.045482e+04,2.355289
8,"MULTIPOLYGON (((-59415.008 6600733.478, -59436...",VP00001370,3.725121e+06,4.851516e+04,7.090917
9,"MULTIPOLYGON (((-52339.912 6629219.972, -52339...",VP00001369,7.580536e+04,4.579078e+03,4.691623


In [63]:
# drop geometry column in duckdb and keep only unique ids in table

# drop geom column
remove_field(db_path, sz_land, "geom")
remove_field(db_path, sz_sea, "geom")

# drop duplicates 
remove_duplicates(db_path, sz_land, "identifikasjon_lokalId")
remove_duplicates(db_path, sz_sea, "identifikasjon_lokalId")

In [64]:
# number of rows in the new table
con.sql(f"SELECT COUNT(*) FROM {sz_land}")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          287 │
└──────────────┘

In [65]:
# number of unique ids in the new table
con.sql(f"SELECT COUNT(DISTINCT identifikasjon_lokalId) FROM {sz_land}")

┌────────────────────────────────────────┐
│ count(DISTINCT identifikasjon_lokalId) │
│                 int64                  │
├────────────────────────────────────────┤
│                                    287 │
└────────────────────────────────────────┘

In [66]:
remove_table(db_path, sz_land_sea)
join_tables_create_new(
    db_path, 
    tbl1=sz_land,
    tbl2=sz_sea,
    id_field="identifikasjon_lokalId",
    new_tbl=sz_land_sea
)

remove_field(db_path, sz_land_sea, "identifikasjon_lokalId_1") 
con.sql("SHOW TABLES;")

┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ ar50_flate           │
│ f_vern_land          │
│ f_vern_land_sjo      │
│ f_vern_land_tmp1     │
│ f_vern_land_tmp2     │
│ f_vern_sjo           │
│ f_vern_sjo_tmp1      │
│ f_vern_sjo_tmp2      │
│ foreslatt_vern_infra │
│ norge_landareal_n50  │
│ norge_ruter_10km     │
│ region_2024          │
│ soner_2017_1km       │
├──────────────────────┤
│       13 rows        │
└──────────────────────┘

In [67]:
con.sql(f"SELECT * FROM {sz_land_sea} LIMIT 5")

┌───────────────────────┬──────────────┬───────────────┬─────────────────┬─────────────┬──────────────┬────────────────┐
│ identifikasjon_loka…  │ landareal_m2 │ landomkrets_m │ land_formindeks │ sjoareal_m2 │ sjoomkrets_m │ sjo_formindeks │
│        varchar        │    float     │     float     │      float      │    float    │    float     │     float      │
├───────────────────────┼──────────────┼───────────────┼─────────────────┼─────────────┼──────────────┼────────────────┤
│ VP00001280            │    238845.64 │      2593.068 │       1.4967536 │        NULL │         NULL │           NULL │
│ VP00000023            │    1549875.6 │      5877.646 │        1.331834 │        NULL │         NULL │           NULL │
│ VP00001140            │    1925020.4 │     7518.5835 │       1.5286689 │        NULL │         NULL │           NULL │
│ VP00001294            │   2312929.75 │     12462.522 │       2.3116379 │        NULL │         NULL │           NULL │
│ VP00001191            │   4269

In [68]:
# number of rows in the new table
con.sql(f"SELECT COUNT(*) FROM {sz_land_sea}")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│          288 │
└──────────────┘

In [69]:
# number of unique ids in the new table
con.sql(f"SELECT COUNT(DISTINCT identifikasjon_lokalId) FROM {sz_land_sea}")

┌────────────────────────────────────────┐
│ count(DISTINCT identifikasjon_lokalId) │
│                 int64                  │
├────────────────────────────────────────┤
│                                    287 │
└────────────────────────────────────────┘

In [70]:
remove_table(db_path, sz_land_vars)
join_tables_create_new(
    db_path, 
    tbl1=tbl_study_area,
    tbl2=sz_land_sea,
    id_field="identifikasjon_lokalId",
    new_tbl=sz_land_vars
)

In [71]:
con.sql("SHOW TABLES;")

┌──────────────────────┐
│         name         │
│       varchar        │
├──────────────────────┤
│ ar50_flate           │
│ f_vern_land          │
│ f_vern_land_sjo      │
│ f_vern_land_tmp1     │
│ f_vern_land_tmp2     │
│ f_vern_land_vars     │
│ f_vern_sjo           │
│ f_vern_sjo_tmp1      │
│ f_vern_sjo_tmp2      │
│ foreslatt_vern_infra │
│ norge_landareal_n50  │
│ norge_ruter_10km     │
│ region_2024          │
│ soner_2017_1km       │
├──────────────────────┤
│       14 rows        │
└──────────────────────┘

In [72]:
con.sql(f"SELECT COUNT(DISTINCT identifikasjon_lokalId) FROM {sz_land_vars}")

┌────────────────────────────────────────┐
│ count(DISTINCT identifikasjon_lokalId) │
│                 int64                  │
├────────────────────────────────────────┤
│                                    289 │
└────────────────────────────────────────┘

**Clean table**

In [73]:
# if null set to 0
con.sql(f"""
    UPDATE {sz_land_vars}
    SET sjoareal_m2 = 0, sjoomkrets_m = 0
    WHERE sjoareal_m2 IS NULL
""")

# if null set to 0
con.sql(f"""
    UPDATE {sz_land_vars}
    SET landareal_m2 = 0, landomkrets_m = 0
    WHERE landareal_m2 IS NULL
""")

# remove cols
remove_field(db_path, sz_land_vars, "identifikasjon_lokalId_1")

# print first 5 rows of the table sz_land_vars
con.sql(f"SELECT * FROM {sz_land_vars} LIMIT 5")

┌──────────────────────┬──────────────────────┬────────────┬───┬─────────────┬──────────────┬────────────────┐
│ identifikasjon_lok…  │ identifikasjon_nav…  │    navn    │ … │ sjoareal_m2 │ sjoomkrets_m │ sjo_formindeks │
│       varchar        │       varchar        │  varchar   │   │    float    │    float     │     float      │
├──────────────────────┼──────────────────────┼────────────┼───┼─────────────┼──────────────┼────────────────┤
│ VP00000313           │ http://www.miljodi…  │ Inngripan  │ … │  4117838.75 │    10731.006 │      1.4917665 │
│ VP00000404           │ http://www.miljodi…  │ Kverndalen │ … │         0.0 │          0.0 │           NULL │
│ VP00000410           │ http://www.miljodi…  │ Båsdalen   │ … │         0.0 │          0.0 │           NULL │
│ VP00000416           │ http://www.miljodi…  │ Rennen     │ … │         0.0 │          0.0 │           NULL │
│ VP00000418           │ http://www.miljodi…  │ Leksa      │ … │         0.0 │          0.0 │           NULL │
├

In [74]:
# to dataframe
df = con.execute(f"SELECT ST_AsText(geom) as geometry, * FROM {sz_land_vars}").fetchdf()

# drop "geom"
df = df.drop(columns=["geom"])

# Convert the geom column to gpd GeoDataFrame
df['geometry'] = df['geometry'].apply(wkt.loads)

# to gpd
gdf = gpd.GeoDataFrame(df, geometry='geometry')
print(f"Number of unique ids: {gdf['identifikasjon_lokalId'].nunique()}")


display(gdf)

Number of unique ids: 289


,geometry,identifikasjon_lokalId,identifikasjon_navnerom,navn,datafangstdato,faktaark,kommune,kvalitet_målemetode,kvalitet_nøyaktighet,verneform,...,area_diff_m2,areal_m2,omkrets_m,formindeks,landareal_m2,landomkrets_m,land_formindeks,sjoareal_m2,sjoomkrets_m,sjo_formindeks
0,"MULTIPOLYGON (((130580.250 7026964.960, 130698...",VP00000313,http://www.miljodirektoratet.no/naturbase,Inngripan,20031217000000,https://faktaark.naturbase.no/?id=VP00000313,1505,NaN,NaN,naturreservat,...,0.000000e+00,4.227805e+06,7814.933105,1.072168,1.099662e+05,2916.072754,2.480641,4117838.75,10731.005859,1.491766
1,"MULTIPOLYGON (((346384.230 7072068.630, 346949...",VP00000404,http://www.miljodirektoratet.no/naturbase,Kverndalen,20000101000000,https://faktaark.naturbase.no/?id=VP00000404,5038,NaN,NaN,naturreservat,...,1.000000e+00,1.099939e+07,14100.687500,1.199364,1.099939e+07,14100.687500,1.199364,0.00,0.000000,NaN
2,"MULTIPOLYGON (((413006.410 7154107.450, 413032...",VP00000410,http://www.miljodirektoratet.no/naturbase,Båsdalen,20000101000000,https://faktaark.naturbase.no/?id=VP00000410,5042,NaN,NaN,naturreservat,...,-1.250000e-01,1.617839e+06,6185.765625,1.371895,1.617839e+06,6185.765625,1.371895,0.00,0.000000,NaN
3,"MULTIPOLYGON (((297926.190 7033095.820, 297935...",VP00000416,http://www.miljodirektoratet.no/naturbase,Rennen,20000101000000,https://faktaark.naturbase.no/?id=VP00000416,5035,NaN,NaN,naturreservat,...,-1.250000e-01,1.811234e+06,7222.564453,1.513907,1.811234e+06,7222.564453,1.513907,0.00,0.000000,NaN
4,"MULTIPOLYGON (((306858.760 7029574.680, 306864...",VP00000418,http://www.miljodirektoratet.no/naturbase,Leksa,20000101000000,https://faktaark.naturbase.no/?id=VP00000418,5035,NaN,NaN,naturreservat,...,5.000000e-01,6.441098e+06,10009.032227,1.112518,6.441098e+06,10009.032227,1.112518,0.00,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,"MULTIPOLYGON (((226914.780 6662878.350, 226896...",VP00001256,http://www.miljodirektoratet.no/naturbase,Melåa,20231110000000,https://faktaark.naturbase.no/?id=VP00001256,3316,NaN,NaN,naturreservat,...,0.000000e+00,4.095157e+05,5771.720215,2.544281,4.095157e+05,5771.720215,2.544281,0.00,0.000000,NaN
286,"MULTIPOLYGON (((66375.300 6956851.290, 66427.7...",VP00001389,http://www.miljodirektoratet.no/naturbase,Skinstadreset,20220208000000,https://faktaark.naturbase.no/?id=VP00001389,1508,NaN,NaN,naturreservat,...,-1.562500e-02,1.808214e+05,2265.909912,1.503186,1.808214e+05,2265.909912,1.503186,0.00,0.000000,NaN
287,"MULTIPOLYGON (((145372.370 7061491.460, 145372...",VP00001032,http://www.miljodirektoratet.no/naturbase,Remman,20200511000000,https://faktaark.naturbase.no/?id=VP00001032,1573,NaN,NaN,marintVerneområde,...,0.000000e+00,3.247644e+07,22787.871094,1.128014,0.000000e+00,0.000000,NaN,0.00,0.000000,NaN
288,"MULTIPOLYGON (((514604.170 8683551.380, 515051...",VP00001251,http://www.miljodirektoratet.no/naturbase,Nedre Adventdalen,20090919000000,https://faktaark.naturbase.no/?id=VP00001251,2111,NaN,NaN,nasjonalparkSvalbard,...,-8.532030e+07,8.532030e+07,94620.875000,2.889720,0.000000e+00,0.000000,NaN,0.00,0.000000,NaN


In [75]:
# Export GDF to file 
filepath = os.path.join(data_path, "processed", "vern_og_bevaring")
gdf.crs = "EPSG:25833"
# Write to existing .geopackage
gdf.to_file(os.path.join(filepath + '.gpkg'), driver='GPKG', layer=sz_land_vars, mode='w')


In [76]:
# gdf to df and remove geom
df = gdf.drop(columns=["geometry"])
# Write to .csv
df.to_csv(os.path.join(data_path, "processed", f'{statistical_zone}_overlapp.csv'))

In [77]:
quit()

### 3. Regional Statistics


Administrative regions:
- Municipalities
- Counties
- Regions 
    - **Nord**: Finnmark, Troms, Nordland
    - **Midt**: Trøndelag, Møre og Romsdal
    - **Sør**: Agder, Vestfold, Telemark
    - **Øst**: Østfold, Akershus, Oslo, Innlandet, Buskerud
    - **Vest**: Vestland, Rogaland


| Protected area | Municipality | County | Region |
|----------------|---------|-------|--------|
| *NaturvernId*  |*kommune*|*fylke*|*region*|
| area A         |         |       |        |

*Regional datasett*:

| Region | Area | Protected Area | Land Protected Area | Marine Protected Area |
|--------|------|----------------|---------------------|-----------------------|
| *Region* | *areal* | *a_vern* | *a_landvern* | *a_marinvern* |
| Nord ||||||


In [ ]:
# DuckDB to dataframe
df = con.execute(f"SELECT ST_AsText(geom) as geometry, * FROM {sz_land_vars}").fetchdf()

# drop "geom"
df = df.drop(columns=["geom"])

# Convert the geom column to gpd GeoDataFrame
df['geometry'] = df['geometry'].apply(wkt.loads)

# to gpd
gdf_statistical_zone = gpd.GeoDataFrame(df, geometry='geometry')
print(f"Number of unique ids: {gdf_statistical_zone['identifikasjon_lokalId'].nunique()}")


display(gdf_statistical_zone)

: 

In [ ]:
# DuckDB to dataframe
df = con.execute(f"SELECT ST_AsText(geom) as geometry, * FROM regions").fetchdf()

# drop "geom"
df = df.drop(columns=["geom"])

# Convert the geom column to gpd GeoDataFrame
df['geometry'] = df['geometry'].apply(wkt.loads)

# to gpd
gdf_region = gpd.GeoDataFrame(df, geometry='geometry')
print(f"Number of unique ids: {gdf_region['identifikasjon_lokalId'].nunique()}")

display(gdf_region)

In [ ]:
import geopandas as gpd

def sum_statistical_zone_in_region(gdf_region, gdf_statistical_zone):
    # Perform a spatial join of the region and protected area GeoDataFrames
    joined = gpd.sjoin(gdf_region, gdf_statistical_zone, how="inner", op='intersects')

    # Group by the region geometry and sum the protected area
    area_sum = joined.groupby(joined.geometry)['areal_m2'].sum()

    # Reset the index of the region GeoDataFrame to its geometry
    gdf_region.set_index(gdf_region.geometry, inplace=True)

    # Add the summed area as a new column to the region GeoDataFrame
    gdf_region['total_statistical_zone'] = area_sum

    # Reset the index of the region GeoDataFrame
    gdf_region.reset_index(drop=True, inplace=True)

    return gdf_region

**Remove temporary tables in duckDB**

In [ ]:
tables = ["vern_land_tmp1", "vern_land_tmp2", "vern_sjo_tmp1", "vern_sjo_tmp2"]
[remove_table(db_path, tbl) for tbl in tables]

###  4. Grid Statistics (10x10 km2)

**Rutenettstatistikk:**

| SSB grid cell | area | Density (land + marine) | Density (land) | 
| --------------|------|-------------------------|----------------|
| *grid_ID* | *grid_areal* | *tetthet_tot_vern* | *tetthet_landvern* | 
| cell 1| 100 km2 | ||





***
$\mathbf{\text{Density}}$<br>
***
The density of protected area per 10x10km (SSB grid) is calculated by dividing the area of the protected area by 100 km^2.

**Density:**&emsp;      $Density = \frac{A}{100 km^2}$



In [ ]:
# Select by region
# SUM protected area in region 
# Area % = SUM(statistical_zone_m2)/(areal_m2) * 100